In [1]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Preparing

In [2]:
site = 3
site_label  = 'AM'

In [15]:
cfg = load_yaml(f'cfg{site}.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

images_path = Path('images')
images_path.mkdir(exist_ok=True)

## Functions

In [16]:
def results2(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'All Pixels',
                'f1': result_df['global_f1'][0],
                'precision': result_df['global_precision'][0],
                'recall': result_df['global_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'No Cloud',
                'f1': result_df['no_cloud_f1'][0],
                'precision': result_df['no_cloud_precision'][0],
                'recall': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'Cloudy',
                'f1': result_df['cloud_f1'][0],
                'precision': result_df['cloud_precision'][0],
                'recall': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )


        
    return pd.DataFrame(exp_results)

## Data preparing

In [20]:
experiments = [
    6,
    9,
    10,
    11,    
    2,
    1,
    3,
    4,
    5,
    7,
    8,
    56,
    59,
    60,
    61,    
    52,
    51,
    53,
    54,
    55,
    57,
    58,
    
]
exp_results = results2(experiments)

In [21]:
exp_results

,name,cloud_cond,f1,precision,recall,group_model
0,CNN-OPT [Free-cloud],All Pixels,0.482011,0.319568,0.980338,CNN
1,CNN-OPT [Free-cloud],No Cloud,0.482013,0.319569,0.980338,CNN
2,CNN-OPT [Free-cloud],Cloudy,0.347826,0.210526,1.000000,CNN
3,CNN-PIX [Free-cloud][12-AverageSAR],All Pixels,0.856597,0.775797,0.956186,CNN
4,CNN-PIX [Free-cloud][12-AverageSAR],No Cloud,0.856597,0.775797,0.956186,CNN
...,...,...,...,...,...,...
61,TRA-FEA [Diverse-cloud][12-AverageSAR][Pre-tra...,No Cloud,0.793246,0.916358,0.699297,SWIN-Unet
62,TRA-FEA [Diverse-cloud][12-AverageSAR][Pre-tra...,Cloudy,0.654289,0.843184,0.534539,SWIN-Unet
63,TRA-DEC [Diverse-cloud][12-AverageSAR][Pre-tra...,All Pixels,0.703137,0.935045,0.563403,SWIN-Unet
64,TRA-DEC [Diverse-cloud][12-AverageSAR][Pre-tra...,No Cloud,0.736799,0.943493,0.604393,SWIN-Unet


In [22]:
exp_results.loc[(exp_results['name'].str.contains('-SAR') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan
exp_results.loc[(exp_results['name'].str.contains('-SAR') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan

cnn_exp_results = exp_results[exp_results['group_model'] == 'CNN']
cnn_max = cnn_exp_results[cnn_exp_results['name'].str.contains('\[Free-cloud\]')]['f1'].max()
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan
cnn_exp_results = cnn_exp_results.replace(' \[Diverse-cloud\]', '-D', regex=True)
cnn_exp_results = cnn_exp_results.replace(' \[Free-cloud\]', '-F', regex=True)
cnn_exp_results = cnn_exp_results.replace('\[Pre-trained\]', '-PT', regex=True)
cnn_exp_results = cnn_exp_results.replace('\[12-AverageSAR\]', '', regex=True)
x_lim1 = 4/11.
x_lim2 = 5/11.

fig = px.bar(cnn_exp_results, x="name", y="f1",
             color='cloud_cond', barmode='group',
             labels = {
                 'f1': 'F1-Score',
                 'name': 'Model',
                 'cloud_cond': 'Cloud Condition'
                 },
             range_y = [0, 1.1],
             height=400,
             width= 800)


fig.add_hrect(y0=0, y1=1, x0 = 0.0, x1 = x_lim1, line_width=0, fillcolor="blue", opacity=0.15, layer = 'below')
fig.add_hrect(y0=0, y1=1, x0 = x_lim2, x1 = 1, line_width=0, fillcolor="red", opacity=0.15, layer = 'below')
fig.add_annotation(text='Cloud-free Models', x=1.5, y=1.05, showarrow = False)
fig.add_annotation(text='Diverse cloud conditions Models', x=7.5, y=1.05, showarrow = False)
fig.add_hline(cnn_max, line_dash = 'dot', annotation_text = 'Better <br> cloud-free <br> Result', annotation_position = 'right')
fig.update_layout(title_text=f'Convolution-Based Models (Site {site})', title_x=0.5)
fig.update_layout(legend=dict(
    x = 1.02,
    y=0.1
))
fig.write_image(f'images/site_{site}_cnn.png')
fig.show()

swin_exp_results = exp_results[exp_results['group_model'] == 'SWIN-Unet']
swin_max = swin_exp_results[swin_exp_results['name'].str.contains('\[Free-cloud\]')]['f1'].max()
swin_exp_results.loc[(swin_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan
swin_exp_results.loc[(swin_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan
swin_exp_results = swin_exp_results.replace(' \[Diverse-cloud\]', '-D', regex=True)
swin_exp_results = swin_exp_results.replace(' \[Free-cloud\]', '-F', regex=True)
swin_exp_results = swin_exp_results.replace('\[Pre-trained\]', '-PT', regex=True)
swin_exp_results = swin_exp_results.replace('\[12-AverageSAR\]', '', regex=True)

fig = px.bar(swin_exp_results, x="name", y="f1",
             color='cloud_cond', barmode='group',
             labels = {
                 'f1': 'F1-Score',
                 'name': 'Model',
                 'cloud_cond': 'Cloud Condition'
                 },
             range_y = [0, 1.1],
             height=400,
             width= 800)

fig.add_hrect(y0=0, y1=1, x0 = 0.0, x1 = x_lim1, line_width=0, fillcolor="blue", opacity=0.15, layer = 'below')
fig.add_hrect(y0=0, y1=1, x0 = x_lim2, x1 = 1, line_width=0, fillcolor="red", opacity=0.15, layer = 'below')
fig.add_annotation(text='Cloud-free Models', x=1.5, y=1.05, showarrow = False)
fig.add_annotation(text='Diverse cloud conditions Models', x=7.5, y=1.05, showarrow = False)
fig.add_hline(swin_max, line_dash = 'dot', annotation_text = 'Better <br> cloud-free <br> Result', annotation_position = 'right')
fig.update_layout(title_text=f'Transformer-Based Models (Site {site})', title_x=0.5)
fig.update_layout(legend=dict(
    x = 1.02,
    y=0.1
))
fig.show()
fig.write_image(f'images/site_{site}_swin.png')